In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sympy as sp
from math import factorial
from tqdm import tqdm
import csv
from scipy.sparse import csr_matrix
from scipy.sparse import identity
from scipy.sparse.linalg import spsolve

In [2]:
np.random.seed(1)
A = 2
M = 6

def getPij(a):
    temp = np.random.rand(A,A)
    return (temp/sum(temp)).T
Pij = getPij(A)
ArrLst = np.random.rand(A)
Beta = 0.3
RhoMtx = np.random.rand(A, A)

Theta = 1.0
Mu =1.0
N = 1

In [27]:
# set the parameters
# get the number of states S
'''
A: number of areas
M: number of total bikes
S: number of total states
Pij: transfering possibility matrix
Beta: broken rate
ArrMtx: arrival rates of each area
Theta: moving rate
Mu: fix rate
RhoMtx: matrix of ride rates
N: number of fix servers
'''

A = 2
M = 6
S = int(factorial(A+A-1+A**2+M)/factorial(A+A-1+A**2)/factorial(M)) * A
print(S)
Pij = [[0.3, 0.7],
       [0.7, 0.3]]
Beta = 0.4
ArrLst = [5.0, 5.0]
Theta = 1.0
Mu = 0.3
RhoMtx = [[1.0, 1.0], 
          [1.0, 1.0]]
N = 1

3432


In [3]:
# generate state dictionary direrctly

State = {}
index = [0]

def temp(x, i, index, l):
    x.append(i)
    generate_R(x.copy(), index, l)
    
def generate_R(s, index, l):
    if len(s)==A+A+A**2 and sum(s)==M:
        State[tuple(s+[l])] = index[0]
        index[0] += 1
    elif len(s)>A+3+A**2 or sum(s)>M:
        return 0
    else:
        for i in range(M+1):
            temp(s.copy(), i, index, l)
            
for l in range(A):
    for i in range(M+1):
        generate_R([i], index, l)
n_state = len(State)
n_queue = A+A+A**2
n_queue, n_state

(8, 3432)

In [4]:
# 使用scipy.sparse.csr_sparse
# generate R matrix

def INi(ni):
    if ni > 0: return 1
    else: return 0
def IL(s):
    if s[A+s[-1]] == 0: return 1
    else: return 0
def IS(l):
    # 判断是不是都是在0到M之间
    if len(l) == sum(list(map(lambda x: M >= x >= 0, l))): return 1
    else: return 0

def arrRateIn(s):
    n, col, val = 0, [], []
    for i in range(A):
        for j in range(A):
            tempS = list(s)
            # Ni, Rij
            y = 2*A + i*A + j
            a1 = tempS[i] = tempS[i] + 1
            a2 = tempS[y] = tempS[y] - 1
            if IS([a1, a2]):
                n += 1
                col.append(State[tuple(tempS)])
                val.append(ArrLst[i]*Pij[i][j])
    #         print(tempS)
    # print(n, col, val)
    # print('arr-----------')
    return n, col, val
            
def backRateIn(s):
    n, col, val = 0, [], []
    for i in range(A):
        for j in range(A):
            tempS = list(s)
            # Ni, Rij
            y = 2*A + j*A + i
            a1 = tempS[i] = tempS[i] - 1
            a2 = tempS[y] = tempS[y] + 1
            if IS([a1, a2]):
                n += 1
                col.append(State[tuple(tempS)])
                val.append(RhoMtx[j][i]*a2*(1-Beta))
    #         print(tempS)
    # print(n, col, val)
    # print('back-----------')
    return n, col, val

def broRateIn(s):
    n, col, val = 0, [], []
    for i in range(A):
        for j in range(A):
            tempS = list(s)
            #Rij, BP
            y = 2*A + j*A + i
            a1 = tempS[y] = tempS[y] + 1
            a2 = tempS[A+i] = tempS[A+i] - 1
            if IS([a1, a2]):
                n += 1
                col.append(State[tuple(tempS)])
                val.append(RhoMtx[j][i]*a1*Beta)
    #         print(tempS)
    # print(n, col, val)
    # print('bro-----------')
    return n, col, val

def fixRateIn(s):
    n, col, val = 0, [], []
    tempS = list(s)
    # RC, DP
    L = s[-1]
    a1 = tempS[A+L] = tempS[A+L] + 1
    a2 = tempS[L] = tempS[L] - 1
    if IS([a1, a2]):
        n += 1
        col.append(State[tuple(tempS)])
        val.append(Mu*min(tempS[A+L], N))
    # print(tempS)
    # print(n, col, val)
    # print('fix-----------')
    return n, col, val

def movRateIn(s):
    n, col, val = 0, [], []
    tempS = list(s)
    index = (s[-1]-1) % A
    tempS[A+index] = 0
    tempS[-1] = index
    if sum(tempS[:-1]) == M:
        n += 1
        col.append(State[tuple(tempS)])
        val.append(Theta)
    # print(tempS)
    # print(n, col, val)
    # print('red-----------')
    return n, col, val
    
def getRateOut(s):
    outRate = 0
    for i in range(A):
        if INi(s[i]):
            outRate += ArrLst[i]
        else: continue
    for i in range(A):
        for j in range(A):
            outRate += RhoMtx[i][j] * s[2*A+i*A+j]
    outRate += Mu*min(s[A+s[-1]], N) + Theta*IL(s)
    # inRate += sum(list(map(lambda a: a[0]*a[1], zip(x,y))))
    return -outRate

def getRateIn(s):
    n, col, val = 0, [], []
    '''
    # customer arrival: arrRateIn
    # ride back: backRateIn
    # ride break down: broRateIn
    # gathering: gathRateIn
    # fixing: fixRateIn
    # redistributing: redRateIn
    '''
    for f in [arrRateIn, backRateIn, broRateIn, fixRateIn, movRateIn]:
        tempN, tempCol, tempVal = f(s)
        n += tempN
        col += tempCol
        val += tempVal
    
    return n, col, val

def generateR():
    #R = csr_matrix((S,S), dtype=np.float)
    Row, Col, Value = [], [], []
    for k, s in enumerate(State):
        '''
        number of row: n
        row number: k
        column number: col
        value: data
        '''
        # 加1
        if k==n_state-1: # collect the last row as a test instance
            tempN, tempCol, tempVal = getRateIn(s)
            tempCol += [k]
            tempVal += [getRateOut(s)]
        else:            # generate the mtx
            # set rate out for state s
            Row += [k]
            Col += [k]
            Value += [getRateOut(s)]

            # set rate in for state s
            tempN, tempCol, tempVal = getRateIn(s)
            Row += [k] * tempN
            Col += tempCol
            Value += tempVal

    Row += [k] * n_state
    Col += list(range(n_state))
    Value += [1] * n_state
    R = csr_matrix((Value, (Row, Col)), dtype=np.float) #.toarray()
    testArr = csr_matrix((tempVal, ([0]*(tempN+1), tempCol)), dtype=np.float)
    return R, testArr
BalanceMtx, testArr = generateR()
    # 原始
#         # set rate out for state s
#         Row += [k]
#         Col += [k]
#         Value += [getRateOut(s)]

#         # set rate in for state s
#         tempN, tempCol, tempVal = getRateIn(s)
#         Row += [k] * tempN
#         Col += tempCol
#         Value += tempVal

#     R = csr_matrix((Value, (Row, Col)), dtype=np.float) #.toarray()
#     return R

# BalanceMtx = generateR()


In [17]:
print(np.linalg.matrix_rank(BalanceMtx.toarray()))
#np.linalg.det(BalanceMtx.toarray())
BalanceMtx.toarray().shape

3432


(3432, 3432)

In [5]:
b = np.array([0]*(n_state-1) + [1])
x = spsolve(BalanceMtx, b)
#b

In [6]:
BalanceMtx.toarray()[-1].dot(x), testArr.toarray().dot(x)

(1.0, array([-1.96920761e-16]))

In [7]:
portionState = {}
for k,s in enumerate(State):
    portionState[s] = x[k]
    #print(s, x[k])

In [33]:
# %store -d State
# %store -d portionState
disState = State
disPorState = portionState
%store disState
%store disPorState

Stored 'disState' (dict)
Stored 'disPorState' (dict)


In [8]:
normalBikes, brokenBikes, idle,  = 0,0,0
for k,s in enumerate(portionState):
    por = portionState[s]
    normalBikes += sum(s[:A]+s[2*A:-1]) * por
    brokenBikes += sum(s[A:2*A]) * por
    if s[A+s[-1]] == 0: idle += por

normalBikes, brokenBikes, idle

(5.931924899668865, 0.06807510033113703, 0.9559669569007838)

In [13]:
sorted(zip(portionState.values(), portionState.keys()), reverse=True)[7000]

(2.0206703688947715e-05, (0, 3, 0, 3, 1, 0, 1, 0, 1))

In [37]:
names=['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 't']
r = pd.read_csv('testPoolCentralSimulation.csv', names=names)

r['sep'] = r.t.diff()
r = r[1:]
r[['a','b']] = r[['a','b']].astype(int)
r = r.drop(columns=['t'])
#r.head()
r.groupby(by=['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i']).sum()

sep
a b c d e f g h i            
0 0 0 0 0 0 0 0 4   35.419437
              1 3  164.238025
              2 2  262.310724
              3 1  232.574894
              4 0   74.160347
...                       ...
3 0 0 0 0 0 1 0 0   38.768088
          1 0 0 0   26.529209
    1 0 0 0 0 0 0  109.387500
  1 0 0 0 0 0 0 0   31.456372
4 0 0 0 0 0 0 0 0   10.835077

[299 rows x 1 columns]